### TODO:

**GRAFICOS**
- [X] criar grafico temporal
- [X] criar grafico diario
- [X] criar distribuicoes de ganho
- [X] criar historico de dividendos
- [X] criar historico de lucro em vendas

**FUNCIONALIDADES**
- [ ] implementar preço açoes compradas no dia
- [ ] implementar subdivisao de acoes ao longo do tempo (qual valor usar?)
- [X] como sera o input dos dados
- [ ] implementar interface com telegram
- [ ] implementar botoes para resultados especificos
- [X] implementar dividendos

In [ ]:
import pandas as pd
import seaborn as sns
from datetime import datetime
from matplotlib import pyplot as plt
import os
from dotenv import load_dotenv, find_dotenv
import dotenv

%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(1, '../src/')


from stockz_ import *
from fiis_bot import *


In [ ]:
# k = FiisBot().update_fiis_dividends()

In [ ]:
st = Stockz()

In [ ]:
st.descriptive_gains()

In [ ]:
st.plot_FIIs_dividends()

In [ ]:
st.plot_historical_gains(300)

In [ ]:
st.plot_daily_gains_distribution(100)

In [ ]:
st.plot_daily_gains(30)

In [ ]:
st.portfolio_tree_plot(300)

In [ ]:
#!conda env export > environment.yml